In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

import tensorflow as tf
import tensorflow_hub as thub
import bert

import pandas as pd

import re

import random

import os
from tqdm import tqdm

import model_utils

/opt/conda/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Initialize BERT model and tokenizer

bert_layer, tokenizer = model_utils.init_bert()

In [3]:
# Initialize Spark context

sc, spark = model_utils.init_spark()

In [4]:
# Read in sarcastic samples, non-sarcastic samples, and the ratio between the two

sarcastic, non_sarcastic, ratio = model_utils.load_data(spark, 
                                                        bucket_name="sarc-bucket-5", 
                                                        dataset="reddit_processed")

In [5]:
generator = model_utils.generate_epoch_df(sarcastic, non_sarcastic, ratio, 5)

In [6]:
X,y = next(generator)

Py4JJavaError: An error occurred while calling o82.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 16 in stage 7.0 failed 4 times, most recent failure: Lost task 16.3 in stage 7.0 (TID 1365, sarc-cluster-w-2.us-central1-c.c.sarcasm-5.internal, executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 366, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 241, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 168, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_len

In [29]:
X

array(DataFrame[context: string], dtype=object)

In [15]:
non_sarc_samp = non_sarcastic.sample(ratio)

In [16]:
epoch_df = sarcastic.union(non_sarc_samp)

In [19]:
funky = model_utils.tokenize_sample
funky

<function model_utils.tokenize_sample(context)>

In [20]:
tokenize_sample_udf = F.udf(funky, ArrayType(IntegerType()))

In [21]:
epoch_df = epoch_df.withColumn("tokens", tokenize_sample_udf(epoch_df.context))

In [11]:
import numpy as np
X = np.array(epoch_df.select('context'))

In [12]:
y = np.array(epoch_df.select('label'))

In [22]:
epoch_df.show()

Py4JJavaError: An error occurred while calling o213.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 1377, sarc-cluster-w-0.us-central1-c.c.sarcasm-5.internal, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 366, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 241, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 168, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with_length(

In [46]:
np.array(epoch_df.select('tokens').collect())

Py4JJavaError: An error occurred while calling o353.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 11 in stage 10.0 failed 4 times, most recent failure: Lost task 11.3 in stage 10.0 (TID 1407, sarc-cluster-w-3.us-central1-c.c.sarcasm-5.internal, executor 13): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 366, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 241, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type, runner_conf)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 168, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 69, in read_command
    command = serializer._read_with

In [47]:
model_utils.tokenize_sample

<function model_utils.tokenize_sample(context)>